In [2]:
# 기본셋팅은 처음에 정리 잘해서 넣어주는게 좋은데
# 본 프로젝트의 목표는 딱히 정리가 아니기 때문에 신경은 안썼다.
# 하지만 다음 프로젝트부터는 목표와 관련없이 정리하는 것을 습관화 하도록 하자.

import pandas as pd
import json

geo_path = './02. skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))

In [4]:
# 분류를 나누어서 저장하기
# 타입, 이름, 주소, 위도, 경도

ulsan_raw = pd.read_excel('ulsan_raw.xlsx')
ulsan_raw.head()

'''
	타입	이름	주소	위도	경도
0	여행지	대왕암공원	울산광역시 동구 등대로 95	NaN	NaN
1	여행지	대왕암 출렁다리	울산광역시 동구 일산동 905	NaN	NaN
2	여행지	울기둥대	울산 동구 등대로 155	NaN	NaN
3	여행지	오영수 문학관	울산광역시 울주군 언양읍 헌양길 280-12	NaN	NaN
4	여행지	언양읍성	울산광역시 울주군 언양읍 동부리 306-1	NaN	NaN
'''

# 여행지, 이름, 주소까지는 일일이 맥의 메모장에 저장으로 1차 작업
# 1차 작업된 것을 엑셀에 넘겨주면서 2차 작업을 완료했다.
# 2차 작업시, 주소작업할때 0층, 00000 박물관 등의 이름은 삭제하고 저장해줬다.
# 그 이유는 스크래핑으로 기계적으로 하기에는 유사한 이름으로 같은게 많았다.
# 주소 부분도 불분명한 주소가 많았어서 구글, 네이버, 그 여행지를 관할하는 관광 사이트에서 주소를 따왔다.

'\n\t타입\t이름\t주소\t위도\t경도\n0\t여행지\t대왕암공원\t울산광역시 동구 등대로 95\tNaN\tNaN\n1\t여행지\t대왕암 출렁다리\t울산광역시 동구 일산동 905\tNaN\tNaN\n2\t여행지\t울기둥대\t울산 동구 등대로 155\tNaN\tNaN\n3\t여행지\t오영수 문학관\t울산광역시 울주군 언양읍 헌양길 280-12\tNaN\tNaN\n4\t여행지\t언양읍성\t울산광역시 울주군 언양읍 동부리 306-1\tNaN\tNaN\n'

In [8]:
import googlemaps

# keys = '구글개인키 입력해주세요'
gmaps = googlemaps.Client(key = keys)

위도 = []
경도 = []

for i, s in ulsan_raw.iterrows():
    # print(i)
    address = s.주소
    
    # lat, lng
    tmp = gmaps.geocode(address, language='ko')
    lat = tmp[0].get('geometry')['location']['lat']
    lng = tmp[0].get('geometry')['location']['lng']
    
    위도.append(lat)
    경도.append(lng)
    
    
# 틀림 포인트가 상당했다.
# 1. iterrows가 아닌 enumate? 그거랑 len, 다 빼고 ulsan_raw로도 써봤는데 길이가 컬럼 기준으로
# 되기 때문에 길이가 4였다. 즉, 위도와 경도를 따내기에는 초기설정이 올바르지 않았다.
# 그래서 iterrows로 rows기준으로 변경하게 되었다.

# 2. 간결하게 하기 위해서 
# s.위도 = lat
# s.경도 = lng
# 라고도 했었는데, 이게 실행엔 문제가 없는데 실질적인 저장이 되지 않더라
# 그래서 반복문 밖에 위도, 경도 빈 리스트 만들어줬고, 반복문 내에도 append형식으로 변경해주었다.

In [9]:
# 우선 생략했지만 위도, 경도를 따로 잘 저장되었는지 확인해주었다.

ulsan_raw.위도= 위도

In [10]:
ulsan_raw.경도 = 경도

In [11]:
# save point

ulsan_raw.to_excel('ulsan.xlsx', index = False)

In [12]:
# 초기 가공 완료된 데이터로 본격적인 시작

ulsan = pd.read_excel('ulsan.xlsx')

In [13]:
for idx, rows in ulsan.iterrows():
    print(rows.타입)
    break
    
# 여행지

여행지


In [16]:
import folium

center= [35.540164, 129.3474133]
my_map = folium.Map(
    location = center,
    zoom_start = 10,
    tiles = 'StamenToner'
)

for idx, rows in ulsan.iterrows():
    
    # 타입 별 색상 설정
    if rows.타입 == '여행지':
        mk_color = 'blue'
        ic_color = 'lightblue'
    elif rows.타입 == '식사':
        mk_color = 'red'
        ic_color = 'lightred'
    elif rows.타입 == '카페':
        mk_color = 'purple'
        ic_color = 'lightpurple'
    elif rows.타입 == '숙소':
        mk_color = 'yellow'
        ic_color = 'lightyellow'
        
    # 지도 마커 생성
    folium.Marker(
        location = [rows['위도'], rows['경도']],
        popup = rows['이름'],
        tolltip = rows['이름'],
        icon = folium.Icon(
            color = mk_color,
            icon_color = ic_color,
            prefix = 'fa'
        )
    ).add_to(my_map)
my_map

/var/folders/5g/z1vg08pd2_97rjb0w3yzhl4m0000gn/T/ipykernel_10524/3650168564.py:31: UserWarning: color argument of Icon should be one of: {'darkpurple', 'lightgreen', 'darkred', 'darkblue', 'red', 'beige', 'black', 'lightgray', 'purple', 'white', 'blue', 'green', 'lightblue', 'lightred', 'cadetblue', 'gray', 'orange', 'pink', 'darkgreen'}.
  icon = folium.Icon(
